# Human Emotion Detection

## Installing required packages

In [1]:
!pip install torch
!pip install opencv-python

## Importing required packages and modules 

In [2]:
import torch
from torch import nn
from torch import optim

import os
import cv2
import numpy as np

from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report

In [4]:
#Creating Testing and Training examples of features and targets from the dataset
X_train, X_test, y_train, y_test = ([], [], [], [])

#Extracting directory name and filenames
for dirname, _, filenames in os.walk('/kaggle/input/emotion-detection-fer/'):
    for file in filenames:
        if file.endswith('.png'):
            #label - test/train
            label = dirname.split('/')[-2]
            #target - emotion category
            category = dirname.split('/')[-1]
            #Adding each image to train and test lists
            if label == 'test':
                X_test.append(cv2.imread(os.path.join(dirname, file), 0))
                y_test.append(category)
            else:
                X_train.append(cv2.imread(os.path.join(dirname, file), 0))
                y_train.append(category)

In [6]:
#Unique target classes
categories = list(set(y_test))
#Assigning each category with a discrete value
def labeling(x):
    for i in range(len(x)):
        x[i] = categories.index(x[i])
    return np.array(x)

y_train = labeling(y_train)
y_test = labeling(y_test)

In [7]:
#Formatting the data into tensors
X = torch.FloatTensor(X_train).view((28709, 1, 48, 48)) #(no.of examples, no.of channels, image size x, image size y)
y = torch.LongTensor(y_train)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  """Entry point for launching an IPython kernel.


In [8]:
X.shape

torch.Size([28709, 1, 48, 48])

## CNN

In [2]:
#defining CNN
model = nn.Sequential(nn.Conv2d(1, 16, 3, 1), #(no.of input channels, no.of outputchannels = filters, size of kernel, stride)
                     nn.ReLU(),
                     nn.MaxPool2d(2,2),#Max-Pooling
                      
                     nn.Conv2d(16, 64, 3, 1),
                     nn.ReLU(),
                     nn.MaxPool2d(2,2),
                     #Flattening the final pooling output and coonecting it to Feed Forward Neural Network
                     nn.Flatten(),
                     nn.Linear(6400, 512), #input layer = 6400
                      nn.ReLU(),
                      nn.Linear(512,512), #hidden layer 1 = 512, hidden layer 2 = 512
                      nn.ReLU(), 
                      nn.Linear(512,512), 
                      nn.ReLU(),
                      nn.Linear(512,512), #hidden layer 3 = 512, hidden layer 4 = 512
                      nn.ReLU(),
                      nn.Linear(512, 7), #Output layer = 7
                      nn.Softmax(dim=1))
print(model)

Sequential(
  (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
  (1): ReLU()
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1))
  (4): ReLU()
  (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Flatten(start_dim=1, end_dim=-1)
  (7): Linear(in_features=6400, out_features=512, bias=True)
  (8): ReLU()
  (9): Linear(in_features=512, out_features=512, bias=True)
  (10): ReLU()
  (11): Linear(in_features=512, out_features=512, bias=True)
  (12): ReLU()
  (13): Linear(in_features=512, out_features=512, bias=True)
  (14): ReLU()
  (15): Linear(in_features=512, out_features=7, bias=True)
  (16): Softmax(dim=1)
)


In [10]:
#For unbalanced dataset, defining class weights
class_weights = compute_class_weight('balanced', classes = np.unique(y_train), y = y_train)

#Defining loss function
loss = nn.CrossEntropyLoss(weight = torch.FloatTensor(class_weights))

#Adam optimizer
sgd = optim.Adam(model.parameters(), lr = 0.001)

#Training CNN
epochs = 100
for epoch in range(epochs):
    output = model(X)
    cost = loss(output, y)
    
    #Setting gradients to 0
    sgd.zero_grad()
    #Back Propagation
    cost.backward()
    sgd.step()
    #Printing training loss for 10 epochs each
    if epoch % (epochs/10) == 0:
        print('Epoch ' + str(epoch) + ' is completed. Training loss is ' + str(cost))
    elif epoch == epochs-1:
        print('Training is completed. Final raining loss is ' + str(cost))
        
X_new = torch.FloatTensor(X_test).view((7178, 1, 48, 48))
y_new = torch.LongTensor(y_test)

#Model testing and evaluation
with torch.no_grad():
    y_pred = model(X_new)
    predictions = y_pred.argmax(dim = 1)
print(predictions)

#Model performance
print(classification_report(predictions, y_new))

Epoch 0 is completed. Training loss is tensor(1.9461, grad_fn=<NllLossBackward0>)
Epoch 10 is completed. Training loss is tensor(2.0225, grad_fn=<NllLossBackward0>)
Epoch 20 is completed. Training loss is tensor(2.0226, grad_fn=<NllLossBackward0>)
Epoch 30 is completed. Training loss is tensor(2.0226, grad_fn=<NllLossBackward0>)
Epoch 40 is completed. Training loss is tensor(2.0226, grad_fn=<NllLossBackward0>)
Epoch 60 is completed. Training loss is tensor(2.0226, grad_fn=<NllLossBackward0>)
Epoch 70 is completed. Training loss is tensor(2.0226, grad_fn=<NllLossBackward0>)
Epoch 90 is completed. Training loss is tensor(2.0226, grad_fn=<NllLossBackward0>)
Training is completed. Final raining loss is tensor(2.0226, grad_fn=<NllLossBackward0>)
tensor([1, 1, 1,  ..., 1, 1, 1])
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.13      0.24      7178
           2       0.00      0.00      0.00         0

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
